In [1]:
import pandas as pd
from pysentimiento import create_analyzer
from tqdm import tqdm
import warnings, re
warnings.filterwarnings("ignore")

In [2]:
# ----------------- CARGA TU ARCHIVO -----------------
df = pd.read_excel(r"C:\Users\9 ----- SIG\Downloads\prueba_comentarios.xlsx")

df = df[['nombre_para', 'respuesta']].copy()
df['respuesta'] = df['respuesta'].astype(str).str.strip()
df = df[df['respuesta'].str.len() > 15]  # quitamos respuestas muy cortas o vacías
print(f"Comentarios válidos para analizar: {len(df)}")

Comentarios válidos para analizar: 29232


In [4]:
# ----------------- MODELOS EN ESPAÑOL -----------------
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")

# Palabras clave que indican crítica (aunque el resto sea positivo)
palabras_criticas = [
    "pero", "aunque", "ojalá", "debería", "a veces", "mucho", "mucha", "muchas", 
    "tarea", "tareas", "trabajo", "pruebas", "exámenes", "difícil", "llega tarde",
    "no explica", "habla rápido", "no alcanza", "poca", "poco", "confuso"
]

def extraer_parte_negativa(texto):
    texto_low = texto.lower()
    # Buscar después de conectores típicos
    for conector in ["pero ", "aunque ", "ojalá ", "sin embargo ", "lastima que ", "lo malo es que "]:
        if conector in texto_low:
            parte = texto_low.split(conector, 1)[1]
            # Tomar hasta el próximo punto o fin
            parte = parte.split('.')[0]
            parte = parte.split(' y ')[0]
            return parte.strip().capitalize()
    
    # Si no hay conector pero hay palabra crítica
    for palabra in palabras_criticas:
        if palabra in texto_low:
            # Tomar la frase alrededor
            inicio = max(0, texto_low.find(palabra) - 30)
            fin = min(len(texto), texto_low.find(palabra) + len(palabra) + 50)
            return texto[inicio:fin].strip() + "..."
    
    return "Ninguno detectado"

In [5]:
# ----------------- ANÁLISIS CORRECTO -----------------
print("Extrayendo el comentario que más baja la nota cuando promedio < 4.20...")
resultados = []

for nombre, grupo in tqdm(df.groupby('nombre_para', sort=False)):
    if len(grupo) == 0: continue
        
    notas = []; pos = neu = neg = criticas = 0
    comentario_peor = None
    peor_nota_individual = 10  # valor alto imposible
    
    for texto in grupo['respuesta']:
        pred = analyzer.predict(texto)
        
        # Sentimiento
        if pred.output == "POS": pos += 1
        elif pred.output == "NEG": neg += 1
        else: neu += 1
            
        # Nota 1-5 del comentario individual
        nota_individual = 1 + 4 * (pred.probas["POS"] + 0.7 * pred.probas["NEU"])
        notas.append(nota_individual)
        
        # Guardamos el comentario con la NOTA MÁS BAJA (el que más daño hace)
        if nota_individual < peor_nota_individual:
            peor_nota_individual = nota_individual
            comentario_peor = texto.strip()
            
        # Conteo de fallas críticas (palabras graves)
        if any(p in texto.lower() for p in ["abandona","nunca viene","insulta","humilla","peor profesor","racista","grita"]):
            criticas += 1
    
    promedio = round(sum(notas)/len(grupo), 2)
    
    # ← REGLA FINAL PERFECTA:
    if promedio < 4.20 and comentario_peor:
        texto_mostrar = comentario_peor[:320]
        if len(comentario_peor) > 320: texto_mostrar += "..."
        falla_columna = texto_mostrar
    else:
        falla_columna = "Sin observaciones relevantes"
    
    resultados.append({
        'Profesor/a'                  : nombre.strip().title(),
        'Valor_Promedio'              : promedio,
        '% Positivo'                  : f"{round(100*pos/len(grupo), 1)}%",
        '% Negativo'                  : f"{round(100*neg/len(grupo), 1)}%",
        '% Neutro'                    : f"{round(100*neu/len(grupo), 1)}%",
        'Falla Crítica'               : "Sí" if criticas >= max(1, len(grupo)*0.12) else "No",
        'Comentario que más baja la nota' : falla_columna
    })

Extrayendo el comentario que más baja la nota cuando promedio < 4.20...


100%|██████████| 1296/1296 [1:16:36<00:00,  3.55s/it]


In [6]:
# ----------------- TABLA FINAL CON LA JOYA -----------------
tabla = pd.DataFrame(resultados)
tabla = tabla.sort_values('Valor_Promedio', ascending=False).reset_index(drop=True)

In [7]:
tabla.tail(10)

,Profesor/a,Valor_Promedio,% Positivo,% Negativo,% Neutro,Falla Crítica,Comentario que más baja la nota
1286,Camilo Rivera,2.00,20.0%,80.0%,0.0%,No,ESTE PROFESOR TIENE CLAROS FAVORITISMOS POR 3 ...
1287,Michel Fajardo,1.88,20.0%,80.0%,0.0%,No,Denasiados trabajos muy robustos que no permit...
1288,Monica Ruiz,1.88,14.3%,71.4%,14.3%,Sí,"Con respecto a la Doctora, hay mucha inconform..."
1289,Laila Bernal,1.81,25.0%,75.0%,0.0%,No,"No me gusto su clase, cuando nos cambiaron de ..."
1290,Angie Tovar,1.81,7.7%,80.8%,11.5%,No,Muy mala profesora no sabe explicar y ni pacie...
1291,Henry Muñoz,1.80,8.3%,83.3%,8.3%,No,El profesor tiene una actitud muy grosera e im...
1292,Daniel Narváez,1.57,0.0%,100.0%,0.0%,No,Una vez más el hecho de que una persona sea bu...
1293,Martha Franco,1.46,0.0%,100.0%,0.0%,No,Si bien la asignatura es en gran medida desarr...
1294,Laura Salazar,1.31,0.0%,100.0%,0.0%,No,"falta mucha organización en clínica, fechas es..."
1295,Martha Gonzalez,1.11,0.0%,100.0%,0.0%,No,"Muy neutral tirando a mala, basa su enseñanza,..."


In [8]:
tabla.to_excel(r"C:\Users\9 ----- SIG\Downloads\analisis_comentario_profesor.xlsx", index=False)